In [ ]:
!pip install pyvi
!pip install nlpaug==1.1.3
!pip install py_vncorenlp
!pip install datasets
!pip install googletrans==4.0.0-rc1
!pip install mtranslate
!pip install transformers
!pip install underthesea
!pip install langdetect
!pip install sentencepiece
!pip install torch

In [ ]:
import pickle
from datasets import Dataset, DatasetDict
dataset_name = "Final_Mixed.pkl"
with open('./data/' + dataset_name, 'rb') as f:
    data = pickle.load(f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
})

In [ ]:
data.set_format(type="pandas")
data_train = data["train"][:]
data.reset_format()
print("Kích thước tập train:", len(data_train))
data_train.head()

Kích thước tập train: 700


,text,label
0,trang thiết bị mới buffet ngon giường hơi nhỏ ...,1
1,beautiful hotels and có phong cách,2
2,tôi và con gái được phục vụ những meals ngon t...,2
3,tôi request phục vụ phòng mấy lần mà đi cả ngà...,0
4,ngoài furama có thể cân nhắc thêm park hyatt n...,1


# Các hàm

In [ ]:
import nlpaug.augmenter.word as naw
import pandas as pd
import py_vncorenlp
import pandas as pd
py_vncorenlp.download_model(save_dir='/content/')
vncore_nlp = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos"], save_dir='/content/')

def word_seg(text):
  output =  vncore_nlp.annotate_text(text)
  df = pd.DataFrame(output[0])
  word_list = list(df['wordForm'])
  return " ".join(word_list)
print("Ví dụ hàm word_seg:",word_seg("người ta có bạn bè nhìn vui vẻ"))

# hàm tiền xử lý
import re
import numpy as np
from underthesea import text_normalize
from pyvi import ViTokenizer
def chuan_hoa_unicode_dau_cau(text):
  return text_normalize(text)

def viet_thuong(text):
	return text.lower()

def chuan_hoa_cau(text):
	text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
	text = re.sub(r'\s+', ' ', text).strip()
	return text

def tien_xu_ly(sen):
  sen = chuan_hoa_unicode_dau_cau(sen)
  sen = viet_thuong(sen)
  sen = chuan_hoa_cau(sen)
  return sen

Ví dụ hàm word_seg: người ta có bạn_bè nhìn vui_vẻ


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_vi2en = AutoTokenizer.from_pretrained("vinai/vinai-translate-vi2en", src_lang="vi_VN")
model_vi2en = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-vi2en")
device_vi2en = torch.device("cuda")
model_vi2en.to(device_vi2en)


def translate_vi2en(vi_texts: str) -> str:
    input_ids = tokenizer_vi2en(vi_texts, padding=True, return_tensors="pt").to(device_vi2en)
    output_ids = model_vi2en.generate(
        **input_ids,
        decoder_start_token_id=tokenizer_vi2en.lang_code_to_id["en_XX"],
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    en_texts = tokenizer_vi2en.batch_decode(output_ids, skip_special_tokens=True)
    return en_texts

# The input may consist of multiple text sequences, with the number of text sequences in the input ranging from 1 up to 8, 16, 32, or even higher, depending on the GPU memory.
vi_texts = ["Cô cho biết: trước giờ tôi không đến phòng tập công cộng, mà tập cùng giáo viên Yoga riêng hoặc tự tập ở nhà.",
            "Khi tập thể dục trong không gian riêng tư, tôi thoải mái dễ chịu hơn.",
            "cô cho biết trước giờ tôi không đến phòng tập công cộng mà tập cùng giáo viên yoga riêng hoặc tự tập ở nhà khi tập thể dục trong không gian riêng tư tôi thoải mái dễ chịu hơn"]
print(translate_vi2en(vi_texts))

["She said: I haven't been to a public gym before, but with my own yoga teacher or at home.", 'When I exercise in a private space, I feel more comfortable.', 'She said: "I\'ve never been to a public gym before, but with my own yoga teacher or at home. When I exercise in a private space, I\'m more comfortable."']


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_en2vi = AutoTokenizer.from_pretrained("vinai/vinai-translate-en2vi", src_lang="en_XX")
model_en2vi = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-en2vi")
device_en2vi = torch.device("cuda")
model_en2vi.to(device_en2vi)


def translate_en2vi(en_texts: str) -> str:
    input_ids = tokenizer_en2vi(en_texts, padding=True, return_tensors="pt").to(device_en2vi)
    output_ids = model_en2vi.generate(
        **input_ids,
        decoder_start_token_id=tokenizer_en2vi.lang_code_to_id["vi_VN"],
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    vi_texts = tokenizer_en2vi.batch_decode(output_ids, skip_special_tokens=True)
    return vi_texts

# The input may consist of multiple text sequences, with the number of text sequences in the input ranging from 1 up to 8, 16, 32, or even higher, depending on the GPU memory.
en_texts = ["I haven't been to a public gym before.",
            "When I exercise in a private space, I feel more comfortable.",
            "i haven't been to a public gym before when i exercise in a private space i feel more comfortable"]
print(translate_en2vi(en_texts))

['Tôi chưa từng đến phòng tập công cộng bao giờ.', 'Khi tôi tập thể dục trong không gian riêng tư, tôi cảm thấy thoải mái hơn.', 'Tôi chưa từng đến phòng tập thể dục công cộng trước đây. Khi tôi tập thể dục trong một không gian riêng tư, tôi cảm thấy thoải mái hơn.']


In [ ]:
print("Ví dụ với hàm translate_en2vi:", translate_vi2en("food"))

Ví dụ với hàm translate_en2vi: ['food']


In [ ]:
import pickle
from datasets import Dataset, DatasetDict
sentences = []
data.set_format(type = "pandas")
train_df = data['train'][:]
dev_df = data['dev'][:]
dev_sentences = list(dev_df['text'])
train_sentences = list(train_df['text'])
data.reset_format()
sentences += dev_sentences + train_sentences
sentences[:5]

['giá cả justifiable',
 'không đeo nameplate',
 'relative hotel cũ kĩ phòng view looking out at the sea cũng không trọn vẹn vì có 1 hành lang to bên ngoài khi 2 tòa nhà bên cạnh mà xây cao thì coi như ko nhìn thấy gì cả',
 'việc tính giá thứ sáu bằng thứ bảy chủ nhật là bất hợp lý',
 'tuy nhiên khách vào xông hơi thì không courteous chút nào']

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
})

In [ ]:
# tách từ trước khi huấn luyện
clean_sentences = []
data_sentences = []
for i in sentences:
  clean = tien_xu_ly(i)
  output =  vncore_nlp.annotate_text(clean)
  try:
    df = pd.DataFrame(output[0])
    word_list = list(df['wordForm'])
    #pos_list = list(df['posTag'])
    clean_sentences.append(clean)
    data_sentences.append(word_list)
  except:
    pass
#data_sentences = [ViTokenizer.tokenize(i).split() for i in clean_sentences]


In [ ]:
print("kích thước sentence", len(sentences))
print("kích thước clean_sentences", len(clean_sentences))
print("kích thước data_sentences", len(data_sentences))


kích thước sentence 800
kích thước clean_sentences 800
kích thước data_sentences 800


In [ ]:
import sklearn.datasets
import re

import nlpaug.augmenter.word as naw
import nlpaug.model.word_stats as nmw

# Train TF-IDF model
#corpus = [" ".join(i) for i in data_sentences]
tfidf_model = nmw.TfIdf()
tfidf_model.train(data_sentences)
tfidf_model.save('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/Mixed_TFIDF')

# Swap word randomly

In [ ]:
swap_aug = naw.RandomWordAug(action="swap", aug_min = 1, aug_max=10, aug_p=0.3)
print("Ví dụ hàm swap_aug:", swap_aug.augment("người ta có bạn_bè nhìn vui_vẻ"))


Ví dụ hàm swap_aug: người ta có bạn_bè vui_vẻ nhìn


In [ ]:
data_aug_swap = data_train.copy()
data_aug_swap['text'] = data_aug_swap['text'].apply(word_seg)
data_aug_swap['text'] = data_aug_swap['text'].apply(swap_aug.augment)

In [ ]:
for i in range(0, 3):
  print(data_train['text'][i])
  print(data_aug_swap['text'][i])

trang thiết bị mới buffet ngon giường hơi nhỏ nếu bạn có 2 bé
mới trang_thiết_bị buffet giường ngon hơi nếu nhỏ bạn có 2 bé
beautiful hotels and có phong cách
Hotels beautiful and có phong_cách
tôi và con gái được phục vụ những meals ngon tận phòng khi có nhu cầu đi bơi ở bể bơi spa gym chỉ với vài bước chân and felt incredibly. convenient when muốn ra biển tắm
tôi và gái con được phục_vụ những meals ngon tận khi có phòng đi bơi nhu_cầu bể_bơi ở spa gym với vài chỉ bước chân and felt incredibly. convenient when ra muốn tắm biển


In [ ]:
import pandas as pd
concat_swap = pd.concat([data_aug_swap, data_train]).sample(frac = 1, random_state = 42)
concat_swap['text'] = concat_swap['text'].apply(lambda x: x.replace('_', " "))
concat_swap = concat_swap.drop_duplicates()
concat_swap['text'] = concat_swap['text'].apply(str.lower)
swap_dict = {'text': list(concat_swap['text']), 'label': list(concat_swap['label'])}
swap_dataset = Dataset.from_dict(swap_dict)
data['aug_swap'] = swap_dataset

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1384
    })
})

# Delete random

In [ ]:
delete_aug = naw.RandomWordAug(action = "delete", aug_min = 1, aug_max=10, aug_p = 0.3)
print("Ví dụ hàm delete_aug:",delete_aug.augment("nhìn người ta có bạn_bè thật là vui_vẻ"))

Ví dụ hàm delete_aug: người ta có bạn_bè thật vui_vẻ


In [ ]:
data_aug_delete = data_train.copy()
data_aug_delete['text'] = data_aug_delete['text'].apply(word_seg)
data_aug_delete['text'] = data_aug_delete['text'].apply(delete_aug.augment)


,text,label
0,trang_thiết_bị mới buffet giường hơi nhỏ bạn có 2,1
1,beautiful hotels có phong_cách,2
2,tôi con được phục_vụ meals ngon có nhu_cầu đi ...,2


In [ ]:
for i in range(0, 3):
  print(data_train['text'][i])
  print(data_aug_delete['text'][i])

trang thiết bị mới buffet ngon giường hơi nhỏ nếu bạn có 2 bé
trang_thiết_bị mới buffet giường hơi nhỏ bạn có 2
beautiful hotels and có phong cách
beautiful hotels có phong_cách
tôi và con gái được phục vụ những meals ngon tận phòng khi có nhu cầu đi bơi ở bể bơi spa gym chỉ với vài bước chân and felt incredibly. convenient when muốn ra biển tắm
tôi con được phục_vụ meals ngon có nhu_cầu đi bơi ở bể_bơi spa gym chỉ vài bước chân incredibly. convenient muốn ra biển tắm


In [ ]:
import pandas as pd
concat_delete = pd.concat([data_aug_delete, data_train]).sample(frac = 1, random_state = 42)
concat_delete['text'] = concat_delete['text'].apply(lambda x: x.replace('_', " "))
concat_delete = concat_delete.drop_duplicates()
concat_delete['text'] = concat_delete['text'].apply(str.lower)
delete_dict = {'text': list(concat_delete['text']), 'label': list(concat_delete['label'])}
delete_dataset = Dataset.from_dict(delete_dict)
data['aug_delete'] = delete_dataset
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1384
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1398
    })
})

# Insert

## Insert synonyms

In [ ]:
import random
from googletrans import Translator
import nltk
from nltk.corpus import wordnet
from mtranslate import translate
# Khởi tạo nltk (nếu chưa được khởi tạo trước đó)
nltk.download('wordnet')

# Khởi tạo Translator
translator = Translator()

def aug_insert(text):
  output =  vncore_nlp.annotate_text(text)
  df = pd.DataFrame(output[0])
  word_list = list(df['wordForm'])
  copy_wordlist = word_list.copy()
  #pos_list = list(df['posTag'])
  numbers = list(range(0, len(word_list)))
  l_max = max(1,int(len(word_list)*0.3))
  # Lặp để chọn ngẫu nhiên và xóa số vừa được chọn ra khỏi danh sách
  l = 0
  while numbers and l < l_max :
      random_number = random.choice(numbers)
      word = copy_wordlist[random_number].replace('_', ' ')
      #print("word:", word)
      en_word = translate_vi2en(word)[0]
      #en_word = translator.translate(word, src='vi', dest='en').text
      synsets = wordnet.synsets(en_word)
      if synsets:
          first_synset = synsets[0]
          en_word = first_synset.lemmas()[0].name()
      vi_word = translate_en2vi(en_word)[0].lower()
      #vi_word = translator.translate(word, src='en', dest='vi').text.lower()
      vi_word = tien_xu_ly(vi_word)
      if vi_word != word:
        #print(vi_word, word)
        idx = random.randint(0, len(word_list) -1)
        word_list.insert(idx, vi_word)
        text = " ".join(word_list)
        l += 1
      numbers.remove(random_number)
  return tien_xu_ly(text)
print("ví dụ hàm aug_insert:")
print("câu ban đầu: ", "Khi tập thể dục trong không gian riêng tư, tôi thoải mái dễ chịu hơn.")
print("câu mới:", aug_insert("Khi tập thể dục trong không gian riêng tư, tôi thoải mái dễ chịu hơn."))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ví dụ hàm aug_insert:
câu ban đầu:  Khi tập thể dục trong không gian riêng tư, tôi thoải mái dễ chịu hơn.
câu mới: khi tập_thể_dục trong không_gian riêng_tư trong các khi nào tôi thoải_mái dễ_chịu hơn


In [ ]:
data_aug_insert = data_train.copy()
#data_aug_insert_vi['text'] = data_aug_insert_vi['text'].apply(aug_insert)
l = 0
for i in range(0, len(data_aug_insert['text'])):
  text = data_aug_insert['text'][i]
  try:
    data_aug_insert.loc[i, 'text'] = aug_insert(text)
  except:
    l += 1
    print("lỗi:", i, l)
    break
  if i % 50 == 0:
    print("[INFOR] processing", i, l)

[INFOR] processing 0 0
[INFOR] processing 50 0
[INFOR] processing 100 0
[INFOR] processing 150 0
[INFOR] processing 200 0
[INFOR] processing 250 0
[INFOR] processing 300 0
[INFOR] processing 350 0
[INFOR] processing 400 0
[INFOR] processing 450 0
[INFOR] processing 500 0
[INFOR] processing 550 0
[INFOR] processing 600 0
[INFOR] processing 650 0


In [ ]:
import pandas as pd
concat_insert = pd.concat([data_aug_insert, data_train]).sample(frac = 1, random_state = 42)
concat_insert['text'] = concat_insert['text'].apply(lambda x: x.replace('_', " "))
concat_insert = concat_insert.drop_duplicates()
concat_insert['text'] = concat_insert['text'].apply(str.lower)
insert_dict = {'text': list(concat_insert['text']), 'label': list(concat_insert['label'])}
insert_dataset = Dataset.from_dict(insert_dict)
data['aug_insert_synonym'] = insert_dataset
import pickle
with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/'+ dataset_name, 'wb') as f:
    pickle.dump(data, f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1366
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1383
    })
    aug_insert_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1334
    })
    aug_insert_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_BERT: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1374
    })
    aug_replace_w2v: Dataset({
        features

In [ ]:
data['aug_insert_synonym']['text'][:5]

['chúng tôi ở đây 2 đêm và hai rất hài lòng của các hài lòng về sự hài lòng đây đây về chúng tôi lựa chọn của mình',
 'về nhà nghỉ ngơi tôi nghỉ ngơi khá chuyến đi hài lòng với chuyến du lịch cùng cả cũng vậy thôi nhà khi nghỉ tôi ở khách sạn chuyến đi này',
 'thái độ phục vụ của các của nhân viên rất tuyệt tuyệt vời',
 'chắc bản thân mình không không ko may không không gặp may',
 'rất xa bố làm ơn ba làm ơn giúp đỡ rất xa có thể nói là khách sạn thái độ phục vụ của nhân viên khách sạn khách sạn kém khách sạn xa so với các khách sạn tư nhân khách sạn 3 4 giúp đỡ sao tại tp hồ chí minh']

## insert word2vec

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec.load("/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/vncore_word2vec_model.bin")
model.wv.most_similar(positive=['khách_sạn'], topn=2)

[('nhà_hàng', 0.9023407697677612), ('ks', 0.8946298360824585)]

In [ ]:
from gensim.models import Word2Vec
import random
model = Word2Vec.load("/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/vncore_word2vec_model.bin")

def aug_insert_w2v(text):
  output =  vncore_nlp.annotate_text(text)
  df = pd.DataFrame(output[0])
  word_list = list(df['wordForm'])
  copy_word_list = word_list.copy()
  pos_list = list(df['posTag'])
  numbers = list(range(0, len(word_list)))
  # Lặp để chọn ngẫu nhiên và xóa số vừa được chọn ra khỏi danh sách
  l = 0
  max_l = max(1,int(len(word_list)*0.3))
  while numbers and l < max_l:
      random_number = random.choice(numbers)
      word = copy_word_list[random_number]
      new_word = model.wv.most_similar(positive=[word], topn=1)[0][0]
      #print(word, new_word)
      new_random_number = random.randint(0, len(word_list) -1)
      word_list.insert(new_random_number, new_word)
      l += 1
      numbers.remove(random_number)
  text = " ".join(word_list)
  return tien_xu_ly(text.replace('_', ' '))

In [ ]:
aug_insert_w2v("tôi rất thích ở khách sạn chất lượng tuyệt vời")

'tôi rất ã thích ở giá thành khách sạn chất lượng tuyệt vời'

In [ ]:
data_aug_insert_w2v = data_train.copy()

In [ ]:

#data_aug_replace_vi['text'] = data_aug_replace_vi['text'].apply(aug_replace)
l = 0
for i in range(0, len(data_aug_insert_w2v['text'])):
  text = data_aug_insert_w2v['text'][i]
  try:
    data_aug_insert_w2v.loc[i, 'text'] = aug_insert_w2v(text)
  except:
    l += 1
    print("Lỗi", i, l)
  if i % 50 == 0:
    print("[INFOR] processing", i, l)

[INFOR] processing 0 0
Lỗi 10 1
Lỗi 18 2
Lỗi 41 3
Lỗi 44 4
[INFOR] processing 50 4
Lỗi 64 5
Lỗi 68 6
Lỗi 85 7
[INFOR] processing 100 7
Lỗi 108 8
Lỗi 118 9
Lỗi 129 10
[INFOR] processing 150 10
Lỗi 158 11
Lỗi 170 12
Lỗi 180 13
[INFOR] processing 200 13
Lỗi 202 14
Lỗi 231 15
Lỗi 239 16
Lỗi 241 17
Lỗi 244 18
[INFOR] processing 250 18
Lỗi 256 19
Lỗi 266 20
Lỗi 276 21
Lỗi 279 22
[INFOR] processing 300 22
Lỗi 310 23
Lỗi 320 24
Lỗi 322 25
[INFOR] processing 350 25
Lỗi 359 26
Lỗi 362 27
Lỗi 376 28
Lỗi 390 29
[INFOR] processing 400 29
[INFOR] processing 450 29
Lỗi 487 30
[INFOR] processing 500 30
Lỗi 541 31
[INFOR] processing 550 31
Lỗi 559 32
[INFOR] processing 600 32
Lỗi 603 33
Lỗi 647 34
[INFOR] processing 650 34
Lỗi 687 35
Lỗi 693 36


In [ ]:
for i in range(0, 5):
  print(data_train['text'][i])
  print(data_aug_insert_w2v['text'][i])

sau 2 ngày đêm ở khách sạn khác chúng tôi quyết định chuyển đến khách sạn camellia thêm 3 ngày đêm nữa
sau 2 ngày đêm ở 3 khách sạn khác chúng tôi quyết định nhà hàng chuyển đến nx khách sạn camellia nhau thêm 3 ngày đêm nữa
khách sạn mới được nâng cấp nội thất mới phòng hơi chật và thấp thông gió chưa tốt nhân viên chu đáo vị trí gần sông hàn và khu trung tâm
khách sạn tận tình mới được cao ms nâng cấp chớp nội thất mới shiper phòng hơi chật và thấp chiến thắng thông gió chưa tốt nhân viên tuy nhiên chu đáo vị trí gần sông hàn và khu trung tâm
nhất là bữa cơm tất niên của gđ mình
mlb nhất là báu vật bữa cơm tất niên của gđ mình
thức ăn tươi ngon rẻ đặc biệt là hải sản tôm cua cá
thức ăn tươi ngon tien rẻ đặc biệt là sườn hải sản tôm cua nấm cá
trước khi đi mình mong muốn có một không gian lãng mạng và ấm cúng ở khách sạn này nhưng khi đến mình nhận ra ngay sai lầm
ã trước khi vài đi mình như vậy miêu tả tuy nhiên mong muốn có một không gian lãng mạng và ấm cúng ở tui khách sạn này như

In [ ]:
#[INFOR] processing 100 0
#[INFOR] processing 250 1
#[INFOR] processing 800 8
#[INFOR] processing 850 8
#[INFOR] processing 1100
# [INFOR] processing 2050
# [INFOR] processing 2200 14

In [ ]:
import pandas as pd
concat_insert_w2v = pd.concat([data_aug_insert_w2v, data_train]).sample(frac = 1, random_state = 42)
concat_insert_w2v['text'] = concat_insert_w2v['text'].apply(lambda x: x.replace('_', " "))
concat_insert_w2v = concat_insert_w2v.drop_duplicates()
concat_insert_w2v['text'] = concat_insert_w2v['text'].apply(str.lower)
insert_w2v_dict = {'text': list(concat_insert_w2v['text']), 'label': list(concat_insert_w2v['label'])}
insert_w2v_dataset = Dataset.from_dict(insert_w2v_dict)
data['aug_insert_w2v'] = insert_w2v_dataset
# replace_vi_dict = {'text': list(data_aug_replace_vi['text']), 'label': list(data_aug_replace_vi['label'])}
# replace_vi_dataset = Dataset.from_dict(replace_vi_dict)
# Mixed['aug_replace_vi_small'] = replace_vi_dataset
import pickle
with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/'+dataset_name, 'wb') as f:
    pickle.dump(data, f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1366
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1383
    })
    aug_insert_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1351
    })
    aug_insert_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_BERT: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1374
    })
    aug_replace_w2v: Dataset({
        features

## insert TFIDF

In [ ]:
# Load TF-IDF augmenter
aug_insert = naw.TfIdfAug(model_path='/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/', aug_p = 0.3, aug_min = 1, action="insert")

def aug_insert_tfidf(text):
  clean = tien_xu_ly(text)
  output =  vncore_nlp.annotate_text(clean)
  try:
    df = pd.DataFrame(output[0])
    word_list = list(df['wordForm'])
    text = " ".join(word_list)
    #print(text)
    text = aug_insert.augment(text)
    #print(text)
  except:
    pass
  return text

In [ ]:
data_aug_insert = data_train.copy()
#data_aug_insert_vi['text'] = data_aug_insert_vi['text'].apply(aug_insert)
l = 0
for i in range(0, len(data_aug_insert['text'])):
  text = data_aug_insert['text'][i]
  try:
    data_aug_insert.loc[i, 'text'] = aug_insert_tfidf(text)
  except:
    l += 1
  if i % 50 == 0:
    if l > 30:
      print("Lỗi gọi hàm dịch")
      break
    else:
      print("[INFOR] processing", i, l)
      l = 0

[INFOR] processing 0 0
[INFOR] processing 50 0
[INFOR] processing 100 0
[INFOR] processing 150 0
[INFOR] processing 200 0
[INFOR] processing 250 0
[INFOR] processing 300 0
[INFOR] processing 350 0
[INFOR] processing 400 0
[INFOR] processing 450 0
[INFOR] processing 500 0
[INFOR] processing 550 0
[INFOR] processing 600 0
[INFOR] processing 650 0


In [ ]:
data_aug_insert[:5]

,text,label
0,sau 2 ngày_đêm ở hải_sản khách_sạn khác chúng_...,1
1,khách_sạn mới mãi được nâng_cấp 12 nội_thất mớ...,1
2,nhất_là xe bữa cơm tất_niên của thơ_mộng gđ mình,1
3,thức_ăn tươi ngon đảm_bảo rẻ còn đặc_biệt là h...,2
4,trước khi đi phải_chăng mình mong_muốn có một ...,0


In [ ]:
import pandas as pd
concat_insert = pd.concat([data_aug_insert, data_train]).sample(frac = 1, random_state = 42)
concat_insert['text'] = concat_insert['text'].apply(lambda x: x.replace('_', " "))
concat_insert = concat_insert.drop_duplicates()
concat_insert['text'] = concat_insert['text'].apply(str.lower)
insert_dict = {'text': list(concat_insert['text']), 'label': list(concat_insert['label'])}
insert_dataset = Dataset.from_dict(insert_dict)
data['aug_insert_tfidf'] = insert_dataset
import pickle
with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/'+ dataset_name, 'wb') as f:
    pickle.dump(data, f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1366
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1383
    })
    aug_insert_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1351
    })
    aug_insert_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_BERT: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1374
    })
    aug_replace_w2v: Dataset({
        features

## insert BERT

In [ ]:
import nlpaug.augmenter.word as naw
from transformers import BertTokenizer, BertForMaskedLM, MarianTokenizer, MarianMTModel
import torch

# Chuẩn bị mô hình BERT và tokenizer cho tiếng Anh
bert_model_name = "bert-base-uncased"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertForMaskedLM.from_pretrained(bert_model_name)

# Tạo đối tượng ContextualWordEmbsAug với BERT tiếng Anh
aug_insert = naw.ContextualWordEmbsAug(
    model_path=bert_model_name, action="insert", aug_min=1, aug_p=0.3
)
# Câu ban đầu tiếng Việt
def aug_insert_BERT(text):
  # Dịch sang tiếng Anh
  en_sentence = translate_vi2en(text)[0]

  # Tạo các câu tăng cường từ câu tiếng Anh đã dịch
  aug_sentence_en = aug_insert.augment(en_sentence, n=1)

  # Dịch các câu tăng cường tiếng Anh sang tiếng Việt
  vi_sentence = translate_en2vi(aug_sentence_en)[0]

  return tien_xu_ly(vi_sentence)

print("Ví dụ với hàm aug_insert_BERT:", aug_insert_BERT("khách sạn đẹp phong cảnh hữu tình nhân viên nhiệt tình vui vẻ"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Ví dụ với hàm aug_insert_BERT: khách sạn rất đẹp phong cảnh quyến rũ với nhân viên quầy bar nhiệt tình vui vẻ


In [ ]:
data_aug_insert= data_train.copy()
count = 0
for i in range(0,len(data_aug_insert)):
  text = data_aug_insert.iloc[i]['text']
  try:
    data_aug_insert.loc[i,'text'] = aug_insert_BERT(text)
  except:
    count +=1
    print("Lỗi", i, count)
    break
  if (i+1) % 20 == 0 or i == 0:
    print("INFOR processing", i)

INFOR processing 0
INFOR processing 19
INFOR processing 39
INFOR processing 59
INFOR processing 79
INFOR processing 99
INFOR processing 119
INFOR processing 139
INFOR processing 159
INFOR processing 179
INFOR processing 199
INFOR processing 219
INFOR processing 239
INFOR processing 259
INFOR processing 279
INFOR processing 299
INFOR processing 319
INFOR processing 339
INFOR processing 359
INFOR processing 379
INFOR processing 399
INFOR processing 419
INFOR processing 439
INFOR processing 459
INFOR processing 479
INFOR processing 499
INFOR processing 519
INFOR processing 539
INFOR processing 559
INFOR processing 579
INFOR processing 599
INFOR processing 619
INFOR processing 639
INFOR processing 659
INFOR processing 679
INFOR processing 699


In [ ]:
for i in range(3):
  print(data_aug_insert['text'][i])
  print(data_train['text'][i])

ngay sau hai hai ngày đêm tại một khách sạn khác chúng tôi chính thức quyết định chuyển đến khách sạn camellia cove thêm ba ngày và mười bốn đêm nữa
sau 2 ngày đêm ở khách sạn khác chúng tôi quyết định chuyển đến khách sạn camellia thêm 3 ngày đêm nữa
nhưng khách sạn mới đã được nâng cấp nội thất mới nhiệt độ phòng nói chung hơi chật và thấp thông gió kém nhân viên chu đáo vị trí trung tâm gần thác sông hàn lớn và trung tâm quận
khách sạn mới được nâng cấp nội thất mới phòng hơi chật và thấp thông gió chưa tốt nhân viên chu đáo vị trí gần sông hàn và khu trung tâm
có lẽ đặc biệt là tại bữa tối chào mừng hàng năm của ông chủ chúng tôi
nhất là bữa cơm tất niên của gđ mình


In [ ]:
import pandas as pd
concat_insert = pd.concat([data_aug_insert, data_train]).sample(frac = 1, random_state = 42)
concat_insert['text'] = concat_insert['text'].apply(lambda x: x.replace('_', " "))
concat_insert = concat_insert.drop_duplicates()
concat_insert['text'] = concat_insert['text'].apply(str.lower)
insert_dict = {'text': list(concat_insert['text']), 'label': list(concat_insert['label'])}
insert_dataset = Dataset.from_dict(insert_dict)
data['aug_insert_BERT'] = insert_dataset
import pickle
with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/'+ dataset_name, 'wb') as f:
    pickle.dump(data, f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1366
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1383
    })
    aug_insert_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1351
    })
    aug_insert_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_BERT: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1374
    })
    aug_replace_w2v: Dataset({
        features

# Replace

In [ ]:
import random
from googletrans import Translator
import nltk
from nltk.corpus import wordnet
from mtranslate import translate
# Khởi tạo nltk (nếu chưa được khởi tạo trước đó)
nltk.download('wordnet')

# Khởi tạo Translator
translator = Translator()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
data_aug_replace = data_train.copy()

In [ ]:
def aug_replace(text):
  output =  vncore_nlp.annotate_text(text)
  df = pd.DataFrame(output[0])
  word_list = list(df['wordForm'])
  pos_list = list(df['posTag'])
  numbers = list(range(0, len(word_list)))
  l = 0
  max_l = int(len(numbers)*0.5)
  # Lặp để chọn ngẫu nhiên và xóa số vừa được chọn ra khỏi danh sách
  while numbers and l < max_l:
      random_number = random.choice(numbers)
      word = word_list[random_number].replace('_', ' ')
      #print(l, word)
      en_word = translate_vi2en(word)[0].lower()
      #print(l, en_word)
      synsets = wordnet.synsets(en_word)
      if synsets:
          first_synset = synsets[0]
          en_word = first_synset.lemmas()[0].name().replace("_", " ")
      #vi_word = translate(en_word, "vi")
      vi_word = translate_en2vi(en_word)[0].lower()
      #print(l, vi_word)
      if vi_word != word:
        word_list[random_number] = vi_word
        l += 1
      numbers.remove(random_number)
  text = " ".join(word_list)
  return tien_xu_ly(text)
print("Test hàm aug_replace synonym:", aug_replace("nhân viên ở đây rất nhiệt tình") )

print("Hàm replace synomym")
for i in range(0, len(data_aug_replace['text'])):
  text = data_aug_replace['text'][i]
  try:
    data_aug_replace.loc[i, 'text'] = aug_replace(text)
  except:
    print("Lỗi gọi hàm:", i)
    break
  if i % 20 == 0:
    print("[INFOR] processing", i)


Test hàm aug_replace synonym: nhân_viên ở lại đây rất nhiệt_tình
Hàm replace synomym
[INFOR] processing 0
[INFOR] processing 20
[INFOR] processing 40
[INFOR] processing 60
[INFOR] processing 80
[INFOR] processing 100
[INFOR] processing 120
[INFOR] processing 140
[INFOR] processing 160
[INFOR] processing 180
[INFOR] processing 200
[INFOR] processing 220
[INFOR] processing 240
[INFOR] processing 260
[INFOR] processing 280
[INFOR] processing 300
[INFOR] processing 320
[INFOR] processing 340
[INFOR] processing 360
[INFOR] processing 380
[INFOR] processing 400
[INFOR] processing 420
[INFOR] processing 440
[INFOR] processing 460
[INFOR] processing 480
[INFOR] processing 500
[INFOR] processing 520
[INFOR] processing 540
[INFOR] processing 560
[INFOR] processing 580
[INFOR] processing 600
[INFOR] processing 620
[INFOR] processing 640
[INFOR] processing 660
[INFOR] processing 680


In [ ]:
test = data_aug_replace.copy()

In [ ]:
for i in range(0, len(data_aug_replace['text'])):
  text = data_aug_replace['text'][i].replace("_", " ")
  word_list = text.split()
  if "đ" in word_list:
    word_list.remove("đ")

  data_aug_replace.loc[i, 'text'] = " ".join(word_list)

In [ ]:
for i in range(5):
  print(data_train['text'][i])
  print(data_aug_replace['text'][i])

sau 2 ngày đêm ở khách sạn khác chúng tôi quyết định chuyển đến khách sạn camellia thêm 3 ngày đêm nữa
mông 2 ngày đêm ở lại lại khách phòng khách sạn cái con người khác chúng tôi quyết định chuyển khoản đến đây đây đây khách phòng khách sạn hoa trà thêm ba làm ơn ngày và đêm ồ rất nhiều nhiều hơn
khách sạn mới được nâng cấp nội thất mới phòng hơi chật và thấp thông gió chưa tốt nhân viên chu đáo vị trí gần sông hàn và khu trung tâm
khách phòng khách sạn mới vâng được rồi lên cao nội bộ thất mới phòng phòng suite hơi nước chặt chẽ và thấp mạng lưới gió chưa đâu đâu tốt số nhân con sóc hãy hãy chu đáo địa điểm tổ chức điểm dừng lại sông hàn và khu vực trung tâm
nhất là bữa cơm tất niên của gđ mình
nhất nó là bữa ăn ăn đi gạo tất niên của gđ đúng thế
thức ăn tươi ngon rẻ đặc biệt là hải sản tôm cua cá
cửa hàng tạp hóa tươi tuyệt vời giá rẻ mạnh mẽ khác biệt nó là bắt đầu cung cấp thuỷ sản tôm cua cá
trước khi đi mình mong muốn có một không gian lãng mạng và ấm cúng ở khách sạn này nhưng 

In [ ]:
import pandas as pd
concat_replace = pd.concat([data_aug_replace, data_train]).sample(frac = 1, random_state = 42)
concat_replace['text'] = concat_replace['text'].apply(lambda x: x.replace('_', " "))
concat_replace = concat_replace.drop_duplicates()
concat_replace['text'] = concat_replace['text'].apply(str.lower)
replace_dict = {'text': list(concat_replace['text']), 'label': list(concat_replace['label'])}
replace_dataset = Dataset.from_dict(replace_dict)
data['aug_replace_synonym'] = replace_dataset
import pickle

with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/'+dataset_name, 'wb') as f:
    pickle.dump(data, f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1369
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1381
    })
    aug_replace: Dataset({
        features: ['text', 'label'],
        num_rows: 1274
    })
    aug_replace_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1345
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1334
    })
    aug_replace_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_BERT: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_backtranslation: Dataset({
        features: ['t

## replace Word2vec

In [ ]:
data_aug_replace = data_train.copy()

In [ ]:
from gensim.models import Word2Vec
import random
model = Word2Vec.load("/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/vncore_word2vec_model.bin")

def aug_replace(text):
  output =  vncore_nlp.annotate_text(text)
  df = pd.DataFrame(output[0])
  word_list = list(df['wordForm'])
  numbers = list(range(0, len(word_list)))
  l = 0
  max_l = int(len(numbers)*0.5)
  # Lặp để chọn ngẫu nhiên và xóa số vừa được chọn ra khỏi danh sách
  while numbers and l < max_l:
      random_number = random.choice(numbers)
      word = word_list[random_number]
      new_word = model.wv.most_similar(positive=[word], topn=1)[0][0]
      #print(word, new_word)
      word_list[random_number] = new_word
      l += 1
  text = " ".join(word_list)
  return tien_xu_ly(text)

print("test hàm aug_replace word2vec", aug_replace("khách sạn này rất đẹp"))
print("Hàm aug replace word2vec")
count = 0
for i in range(0, len(data_aug_replace['text'])):
  text = data_aug_replace['text'][i]
  try:
    data_aug_replace.loc[i, 'text'] = aug_replace(text)
  except:
    count +=1
    print("Lỗi gọi hàm:", i, count)
  if i % 20 == 0:
    print("[INFOR] processing", i)

test hàm aug_replace word2vec khách_sạn như_vậy cực_kì đẹp
Hàm aug replace synonym
Lỗi gọi hàm: 0 1
[INFOR] processing 0
Lỗi gọi hàm: 10 2
Lỗi gọi hàm: 18 3
[INFOR] processing 20
[INFOR] processing 40
Lỗi gọi hàm: 44 4
[INFOR] processing 60
Lỗi gọi hàm: 68 5
[INFOR] processing 80
Lỗi gọi hàm: 84 6
Lỗi gọi hàm: 88 7
[INFOR] processing 100
Lỗi gọi hàm: 117 8
Lỗi gọi hàm: 118 9
[INFOR] processing 120
Lỗi gọi hàm: 124 10
Lỗi gọi hàm: 128 11
Lỗi gọi hàm: 129 12
Lỗi gọi hàm: 140 13
[INFOR] processing 140
Lỗi gọi hàm: 158 14
[INFOR] processing 160
Lỗi gọi hàm: 170 15
Lỗi gọi hàm: 180 16
[INFOR] processing 180
[INFOR] processing 200
Lỗi gọi hàm: 202 17
[INFOR] processing 220
Lỗi gọi hàm: 231 18
Lỗi gọi hàm: 239 19
[INFOR] processing 240
Lỗi gọi hàm: 241 20
Lỗi gọi hàm: 244 21
Lỗi gọi hàm: 256 22
[INFOR] processing 260
Lỗi gọi hàm: 262 23
Lỗi gọi hàm: 276 24
[INFOR] processing 280
[INFOR] processing 300
Lỗi gọi hàm: 320 25
[INFOR] processing 320
Lỗi gọi hàm: 322 26
Lỗi gọi hàm: 329 27
Lỗi gọi h

In [ ]:
for i in range(3):
  print(data_train['text'][i])
  print(data_aug_replace['text'][i])

sau 2 ngày đêm ở khách sạn khác chúng tôi quyết định chuyển đến khách sạn camellia thêm 3 ngày đêm nữa
sau 2 ngày đêm ở khách sạn khác chúng tôi quyết định chuyển đến khách sạn camellia thêm 3 ngày đêm nữa
khách sạn mới được nâng cấp nội thất mới phòng hơi chật và thấp thông gió chưa tốt nhân viên chu đáo vị trí gần sông hàn và khu trung tâm
khách_sạn ms được cập nội_thất mới phòng hơi thúi và thấp chiến_thắng gió chưa_thể tốt nhân_viên chu_đáo vị_trí 12 bờ hàn và vị_trí trung_tâm
nhất là bữa cơm tất niên của gđ mình
mlb hồi cơm lứa của gđ mình


In [ ]:
import pandas as pd
concat_replace = pd.concat([data_aug_replace, data_train]).sample(frac = 1, random_state = 42)
concat_replace['text'] = concat_replace['text'].apply(lambda x: x.replace('_', " "))
concat_replace = concat_replace.drop_duplicates()
concat_replace['text'] = concat_replace['text'].apply(str.lower)
replace_dict = {'text': list(concat_replace['text']), 'label': list(concat_replace['label'])}
replace_dataset = Dataset.from_dict(replace_dict)
data['aug_replace_w2v'] = replace_dataset
import pickle

with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/'+dataset_name, 'wb') as f:
    pickle.dump(data, f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1369
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1381
    })
    aug_replace: Dataset({
        features: ['text', 'label'],
        num_rows: 1274
    })
    aug_replace_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1334
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1334
    })
    aug_replace_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_BERT: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_backtranslation: Dataset({
        features: ['t

## replace TFIDF

In [ ]:
data_aug_replace = data_train.copy()

In [ ]:
# Load TF-IDF augmenter
nlp_replace = naw.TfIdfAug(model_path='/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/', aug_p = 0.5, aug_min = 1, action="substitute")

def aug_replace(text):
  clean = tien_xu_ly(text)
  output =  vncore_nlp.annotate_text(clean)
  df = pd.DataFrame(output[0])
  word_list = list(df['wordForm'])
  text = " ".join(word_list)
  text = nlp_replace.augment(text)
  return text
print("Test hàm aug_replace TFIDF:", aug_replace("nhân viên ở đây rất nhiệt tình") )

print("Hàm replace TFIDF")
for i in range(0, len(data_aug_replace['text'])):
  text = data_aug_replace['text'][i]
  try:
    data_aug_replace.loc[i, 'text'] = aug_replace(text)
  except:
    print("Lỗi gọi hàm:", i)
    break
  if i % 20 == 0:
    print("[INFOR] processing", i)

Test hàm aug_replace TFIDF: nước ở đây rất điểm_tâm
Hàm replace TFIDF
[INFOR] processing 0
[INFOR] processing 20
[INFOR] processing 40
[INFOR] processing 60
[INFOR] processing 80
[INFOR] processing 100
[INFOR] processing 120
[INFOR] processing 140
[INFOR] processing 160
[INFOR] processing 180
[INFOR] processing 200
[INFOR] processing 220
[INFOR] processing 240
[INFOR] processing 260
[INFOR] processing 280
[INFOR] processing 300
[INFOR] processing 320
[INFOR] processing 340
[INFOR] processing 360
[INFOR] processing 380
[INFOR] processing 400
[INFOR] processing 420
[INFOR] processing 440
[INFOR] processing 460
[INFOR] processing 480
[INFOR] processing 500
[INFOR] processing 520
[INFOR] processing 540
[INFOR] processing 560
[INFOR] processing 580
[INFOR] processing 600
[INFOR] processing 620
[INFOR] processing 640
[INFOR] processing 660
[INFOR] processing 680


In [ ]:
for i in range(3):
  print(data_train['text'][i])
  print(data_aug_replace['text'][i])

sau 2 ngày đêm ở khách sạn khác chúng tôi quyết định chuyển đến khách sạn camellia thêm 3 ngày đêm nữa
sau chú_thích cách tránh xuân khác chúng_tôi quyết_định trò đến khách_sạn bé thêm lót ngày_đêm sữa
khách sạn mới được nâng cấp nội thất mới phòng hơi chật và thấp thông gió chưa tốt nhân viên chu đáo vị trí gần sông hàn và khu trung tâm
khách_sạn vướng được nhà_cung_cấp nội_thất nệp phòng hơi chật cảnh thấp quý_vị gió ocean mõi nhân_viên chu_đáo tương_tự gần sông hàn và phía đến_nỗi
nhất là bữa cơm tất niên của gđ mình
nhất_là thiêu_thân cơm tất_niên ngọc_lan gđ gái


In [ ]:
import pandas as pd
concat_replace = pd.concat([data_aug_replace, data_train]).sample(frac = 1, random_state = 42)
concat_replace['text'] = concat_replace['text'].apply(lambda x: x.replace('_', " "))
concat_replace = concat_replace.drop_duplicates()
concat_replace['text'] = concat_replace['text'].apply(str.lower)
replace_dict = {'text': list(concat_replace['text']), 'label': list(concat_replace['label'])}
replace_dataset = Dataset.from_dict(replace_dict)
data['aug_replace_tfidf'] = replace_dataset
import pickle

with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/'+dataset_name, 'wb') as f:
    pickle.dump(data, f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1366
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1383
    })
    aug_insert_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1351
    })
    aug_insert_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_BERT: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1374
    })
    aug_replace_w2v: Dataset({
        features

## replace BERT

In [ ]:
data_aug_replace = data_train.copy()

In [ ]:
nlp_replace = naw.ContextualWordEmbsAug(
    model_path=bert_model_name, action="substitute", aug_min=1, aug_p=0.5
)
def aug_replace(text):
  clean = tien_xu_ly(text)
  en_sentence = translate_vi2en(clean)[0]
  #print(en_sentence)
  aug_sentence = nlp_replace.augment(en_sentence)
  #print(aug_sentence)
  vi_sentence = translate_en2vi(aug_sentence)[0]
  #print(vi_sentence)
  return tien_xu_ly(vi_sentence)
print("Test hàm aug_replace BERT:", aug_replace("nhân viên ở đây rất nhiệt tình") )

print("Hàm replace BERT")
for i in range(0, len(data_aug_replace['text'])):
  text = data_aug_replace['text'][i]
  try:
    data_aug_replace.loc[i, 'text'] = aug_replace(text)
  except:
    print("Lỗi gọi hàm:", i)
    break
  if i % 20 == 0:
    print("[INFOR] processing", i)

Test hàm aug_replace BERT: thằng bé có vẻ vô cùng hào hứng
Hàm replace BERT
[INFOR] processing 0
[INFOR] processing 20
[INFOR] processing 40
[INFOR] processing 60
[INFOR] processing 80
[INFOR] processing 100
[INFOR] processing 120
[INFOR] processing 140
[INFOR] processing 160
[INFOR] processing 180
[INFOR] processing 200
[INFOR] processing 220
[INFOR] processing 240
[INFOR] processing 260
[INFOR] processing 280
[INFOR] processing 300
[INFOR] processing 320
[INFOR] processing 340
[INFOR] processing 360
[INFOR] processing 380
[INFOR] processing 400
[INFOR] processing 420
[INFOR] processing 440
[INFOR] processing 460
[INFOR] processing 480
[INFOR] processing 500
[INFOR] processing 520
[INFOR] processing 540
[INFOR] processing 560
[INFOR] processing 580
[INFOR] processing 600
[INFOR] processing 620
[INFOR] processing 640
[INFOR] processing 660
[INFOR] processing 680


In [ ]:
for i in range(3):
  print(data_train['text'][i])
  print(data_aug_replace['text'][i])

sau 2 ngày đêm ở khách sạn khác chúng tôi quyết định chuyển đến khách sạn camellia thêm 3 ngày đêm nữa
sau hai tiếng đồng hồ và đêm quan sát khách sạn của họ jack quyết định chống lại một quán bar hoa trà khác trong ba giờ nữa cả đêm
khách sạn mới được nâng cấp nội thất mới phòng hơi chật và thấp thông gió chưa tốt nhân viên chu đáo vị trí gần sông hàn và khu trung tâm
khách sạn đã từng bước mở rộng nội thất mới phòng nghỉ tuy chật nhưng thoải mái thông gió kém nhân viên chu đáo vị trí trung tâm làng công nghiệp hàn hàn
nhất là bữa cơm tất niên của gđ mình
đặc biệt là tại cuộc họp tiếp theo của cha tôi


In [ ]:
import pandas as pd
concat_replace = pd.concat([data_aug_replace, data_train]).sample(frac = 1, random_state = 42)
concat_replace['text'] = concat_replace['text'].apply(lambda x: x.replace('_', " "))
concat_replace = concat_replace.drop_duplicates()
concat_replace['text'] = concat_replace['text'].apply(str.lower)
replace_dict = {'text': list(concat_replace['text']), 'label': list(concat_replace['label'])}
replace_dataset = Dataset.from_dict(replace_dict)
data['aug_replace_BERT'] = replace_dataset
import pickle

with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/'+dataset_name, 'wb') as f:
    pickle.dump(data, f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1369
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1381
    })
    aug_replace: Dataset({
        features: ['text', 'label'],
        num_rows: 1274
    })
    aug_replace_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1334
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1334
    })
    aug_replace_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_BERT: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_backtranslation: Dataset({
        features: ['t

# Dịch ngược

In [ ]:
with open('/content/result.txt', 'r', encoding='utf-8-sig') as f:
  result = f.read().split('\n')
len(result)

285

In [ ]:
data_aug_BackTranslation = data_train.copy()
for i in range(285, 700):
  try:
    sentence_list = list(data_aug_BackTranslation['text'][i])
    en_list = translate_vi2en(sentence_list)
    vi_list = translate_en2vi(en_list)
    result += vi_list
  except:
    print("Lỗi", i)
    break
  if i % 50 ==0:
    print("INFOR processing", i)

Lỗi 289


In [ ]:
len(result)

285

In [ ]:
result[0:5]

['Sau hai ngày đêm ở một khách sạn khác, chúng tôi quyết định chuyển đến khách sạn Camellia thêm ba ngày đêm nữa.',
 'Khách sạn đã được nâng cấp, nội thất mới, phòng hơi chật và thấp, thông gió kém, nhân viên chu đáo, vị trí gần sông Hàn và trung tâm.',
 'Đặc biệt là tại bữa tối thường niên của sếp chúng ta.',
 'Thực phẩm tươi sống có giá rẻ, đặc biệt là hải sản (tôm, cua, cá).',
 'Trước khi rời đi, tôi muốn có một bầu không khí lãng mạn và ấm cúng tại khách sạn này, nhưng khi tôi đến, tôi ngay lập tức nhận ra sai lầm.']

In [ ]:
data['aug_backtranslation']['text'][:5]

['chúng tôi ở đây hai đêm và rất hài lòng với sự lựa chọn của chúng tôi',
 'tôi rất hân hạnh được đi du lịch với gia đình tôi trong khi tôi đang ở khách sạn này',
 'anh cũng không có gì',
 'tôi đoán tôi đã không nhận được may mắn bản thân mình',
 'cứ cho là thái độ của nhân viên khách sạn ở xa khách sạn tư nhân']

In [ ]:
import pandas as pd
concat_BackTranslation = pd.concat([data_aug_BackTranslation, data_train]).sample(frac = 1, random_state = 42)
concat_BackTranslation['text'] = concat_BackTranslation['text'].apply(lambda x: x.replace('_', " "))
concat_BackTranslation = concat_BackTranslation.drop_duplicates()
concat_BackTranslation['text'] = concat_BackTranslation['text'].apply(str.lower)
BackTranslation_dict = {'text': list(concat_BackTranslation['text']), 'label': list(concat_BackTranslation['label'])}
BackTranslation_dataset = Dataset.from_dict(BackTranslation_dict)
data['aug_backtranslation'] = BackTranslation_dataset

import pickle

with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/' + dataset_name, 'wb') as f:
    pickle.dump(data, f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1369
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1381
    })
    aug_insert: Dataset({
        features: ['text', 'label'],
        num_rows: 1372
    })
    aug_replace: Dataset({
        features: ['text', 'label'],
        num_rows: 1274
    })
    aug_replace_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1345
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1335
    })
    aug_replace_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_BERT: Dataset({
        features: ['text', 'la

# Check data

In [ ]:
import pickle
from datasets import Dataset, DatasetDict
#dataset_name = "Final_VietNam.pkl"
with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/' + dataset_name, 'rb') as f:
    data = pickle.load(f)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 700
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 200
    })
    aug_swap: Dataset({
        features: ['text', 'label'],
        num_rows: 1376
    })
    aug_delete: Dataset({
        features: ['text', 'label'],
        num_rows: 1383
    })
    aug_insert_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_w2v: Dataset({
        features: ['text', 'label'],
        num_rows: 1351
    })
    aug_insert_tfidf: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_insert_BERT: Dataset({
        features: ['text', 'label'],
        num_rows: 1387
    })
    aug_replace_synonym: Dataset({
        features: ['text', 'label'],
        num_rows: 1374
    })
    aug_replace_w2v: Dataset({
        features

In [ ]:
data.keys()

dict_keys(['train', 'dev', 'test', 'aug_swap', 'aug_delete', 'aug_replace', 'aug_replace_w2v', 'aug_insert_w2v', 'aug_replace_tfidf', 'aug_replace_BERT', 'aug_backtranslation', 'aug_insert_synonym', 'aug_insert_tfidf', 'aug_insert_BERT', 'aug_replace_synonym'])

In [ ]:
new_key = ['train', 'dev', 'test', 'aug_swap', 'aug_delete', 'aug_insert_synonym', 'aug_insert_w2v', 'aug_insert_tfidf', 'aug_insert_BERT', 'aug_replace_synonym', 'aug_replace_w2v', 'aug_replace_tfidf', 'aug_replace_BERT', 'aug_backtranslation']

In [ ]:
len(new_key)

14

In [ ]:
len(list(data.keys()))

15

In [ ]:
new_data = {key: data[key] for key in new_key}

In [ ]:
new_data

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 700
 }),
 'dev': Dataset({
     features: ['text', 'label'],
     num_rows: 100
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 200
 }),
 'aug_swap': Dataset({
     features: ['text', 'label'],
     num_rows: 1369
 }),
 'aug_delete': Dataset({
     features: ['text', 'label'],
     num_rows: 1381
 }),
 'aug_insert_synonym': Dataset({
     features: ['text', 'label'],
     num_rows: 1366
 }),
 'aug_insert_w2v': Dataset({
     features: ['text', 'label'],
     num_rows: 1334
 }),
 'aug_insert_tfidf': Dataset({
     features: ['text', 'label'],
     num_rows: 1387
 }),
 'aug_insert_BERT': Dataset({
     features: ['text', 'label'],
     num_rows: 1387
 }),
 'aug_replace_synonym': Dataset({
     features: ['text', 'label'],
     num_rows: 1374
 }),
 'aug_replace_w2v': Dataset({
     features: ['text', 'label'],
     num_rows: 1334
 }),
 'aug_replace_tfidf': Dataset({
     features: ['text', 'label

In [ ]:
for data_name in new_key:
  print(data_name, new_data[data_name]['text'][0:5])

train ['sau 2 ngày đêm ở khách sạn khác chúng tôi quyết định chuyển đến khách sạn camellia thêm 3 ngày đêm nữa', 'khách sạn mới được nâng cấp nội thất mới phòng hơi chật và thấp thông gió chưa tốt nhân viên chu đáo vị trí gần sông hàn và khu trung tâm', 'nhất là bữa cơm tất niên của gđ mình', 'thức ăn tươi ngon rẻ đặc biệt là hải sản tôm cua cá', 'trước khi đi mình mong muốn có một không gian lãng mạng và ấm cúng ở khách sạn này nhưng khi đến mình nhận ra ngay sai lầm']
dev ['vui vẻ và hấp dẫn khách sạn có quầy bar ở lobby rất ấm cúng và nhẹ nhàng với nhiều loại đồ uống không khí quầy bar tạo cho khách cảm giác hết sức thư thái và tĩnh mịch dù cho chỉ cần nhìn ra ngoài đã thấy quang cảnh phố tây tấp nập ở nha trang', 'tuy nhiên nếu có dịp quay lại tôi vẫn thích được ở đây', 'cám ơn', 'có bức tường rất to chắn trước cửa sổ muốn ngắm hồ phải bắc ghế', 'mặt tiền khách sạn hơi nhỏ và bên cạnh đang sửa chữa nên có một chút gì ảnh hưởng đến mỹ quan của khách sạn nhưng không sao có thể bỏ qua

In [ ]:
with open('/content/drive/MyDrive/NLP/Data augmentation/data/Code-mixed dataset with label/'+dataset_name, 'wb') as f:
    pickle.dump(new_data, f)